### Attention2.md 에 있는 순서대로 진행해보자 !

# -------- Step 1 ---------

### Encoder

In [1]:
#한국어 영어 번역 데이터셋 만들기
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import re

# 실제 프로젝트 AI hub 기타등등..

korean_sentences = [
    "안녕하세요",
    "오늘 날씨가 좋아요",
    "드라마 보고 있어",
    "이것은 사과입니다",
    "고양이가 자고 있어요",
    "내일 비가 올까요",
    "저는 커피를 좋아해요",
    "그는 의사입니다",
    "이 책은 재미있어요",
    "오늘 몸이 안 좋아"
]

english_sentences = [
    "hello",
    "the weather is nice today",
    "I’m watching a drama",
    "this is an apple",
    "the cat is sleeping",
    "will it rain tomorrow",
    "i like coffee",
    "he is a doctor",
    "this book is interesting",
    "I don’t feel well today"
]

# 텍스트 전처리
# uicode 정규화 , 툭수문자처리
# NFD Normalization Form Decomposition -> 분해 가능한 모든 문자를 분해한다.
unicodedata.normalize('NFD',"e ' ")

"e ' "

$\acute{e}$

단일문자 U+00E9

$e$ + $\acute{}$

$e$ + 악센트가 결합이 된 형태

Unicode 정규화란 ?  같은 문장처럼 보이지만 내부적으로 다른 바이트 조합을 가지는 경우가 있는데 이걸 동일한 내부 표현으로 바꿔주는 과정

In [2]:
def preprocess_sentence(sentence , is_korean = False):
    '''
    Unicode 정규화 , 특수문자 처리
    Args : 
        sentence : 원본
        is_korean : 한국어 여부
    return:
        전처리된 문장
    '''
    sentence = unicodedata.normalize('NFD' , sentence)
    if not is_korean:
        sentence = sentence.lower()
    sentence = sentence.strip() # .strip() 좌우공백제거
    # 안녕하세요! -> 안녕하세요 !  정규식으로 특수문자 전 후에 공백 추가
    # r"[?.!,]"   문자 중에 ?.!, 특수문자가 나오면 문자를 그룹으로 캡쳐
    # r"\1"  캡쳐한 문자 (\1) 앞뒤로 공백을 하나씩 넣는다.
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]', " ", sentence)
    # 시작 종료 토큰 추가
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

korean_processed = [preprocess_sentence(kor) for kor in korean_sentences]
english_processed = [preprocess_sentence(eng) for eng in english_sentences]

# 문장을 정수 시퀀스로 변환
# 3 토크나이저

def create_tokenizer(sentence):
    '''
    단어를 정수 인덱스로 변환
    Vocabulary 구축 , word to index mapping
    Args:
        sentence : 문장 리스트
    returns:
        Tokenizer : keras Tokenizer 객체
    '''
    tokenizer = Tokenizer(
        filters='' , # 필터 비활성화
        oov_token = '<unk>'
    )
    # 모든 문장으로 단어사전 구축
    tokenizer.fit_on_texts(sentence)
    return tokenizer

# 한국어 영어 각가의 토크나이져 생성
korean_tokenizer = create_tokenizer(korean_processed)
english_tokenizer = create_tokenizer(english_processed)

# 단어사전의 크기를 확인
korean_vocab_size = len(korean_tokenizer.word_index)+1 # oov_token = '<unk>' 때문에 플러스 1을 해준거임
english_vocab_size = len(english_tokenizer.word_index)+1
print(f'한국어 사전 크기 : {korean_vocab_size}')
print(f'영어 사전 크기 : {english_vocab_size}')

# 정수 시퀀스 변경
def encode_sentences(tokenizer , sentneces , maxlen):
    '''
    문장을 고정 길이의 정수 시퀀스로 변환
    padding : 짧은 문장을 동일 길이로 맞추는
    Args : 
        Tokenizer
        sentence : 문장리스트
        maxlen : 가장 긴 문장
    returns:
        패딩된 정수 시퀀스 배열 tf의 pad_sequence
    '''
    sequence = tokenizer.texts_to_sequences(sentneces) # [i love you] -> [1,5,7]
    
    # 길이맞추기 (패딩추가)
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequence, 
        maxlen = maxlen,
        padding='post'
    )
    return padded

# 최대 시퀀스의 길이 결정 (가장 긴 문장을 기준으로!)
max_korean_len = max( len(s.split()) for s in korean_processed )
max_english_len = max( len(s.split()) for s in english_processed )
print(f'시퀀스의 최대길이')
print(f'한국어 : {max_korean_len}')
print(f'영어 : {max_english_len}')

# 인코딩 수행
korean_tensor = encode_sentences(korean_tokenizer , korean_processed , max_korean_len)
english_tensor = encode_sentences(english_tokenizer , english_processed , max_english_len)
print(f'인코딩 두번째 문장')

print(f'한국어 두번째 문장 : {korean_processed[1]}')
print(f'한국어 인코딩 문장 : {korean_tensor[1]}')

print(f'영어 두번째 문장 : {english_processed[1]}')
print(f'영어 인코딩 문장 : {english_tensor[1]}')



BUFFER_SIZE = len(korean_tensor)
BATCH_SIZE = 2

#tensorflow Dataset 객체
# 배치처리와 셔플

dataset = tf.data.Dataset.from_tensor_slices( (korean_tensor , english_tensor) )
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE , drop_remainder=True)
print('데이터셋 준비완료')
print(f'전체 샘플 수 : {len(korean_tensor)}')
print(f'배치 사이즈 : {BATCH_SIZE}')
print(f'배치수 : {len(korean_tensor) // BATCH_SIZE}')


한국어 사전 크기 : 30
영어 사전 크기 : 33
시퀀스의 최대길이
한국어 : 6
영어 : 7
인코딩 두번째 문장
한국어 두번째 문장 : <start> 오늘 날씨가 좋아요 <end>
한국어 인코딩 문장 : [2 4 6 7 3 0]
영어 두번째 문장 : <start> the weather is nice today <end>
영어 인코딩 문장 : [ 2  5 11  4 12  6  3]
데이터셋 준비완료
전체 샘플 수 : 10
배치 사이즈 : 2
배치수 : 5


# ------- step2 ---------

### seq2seq 구조

Encoder

    - 입력문장 -> Embedding -> LSTM + Hidden States

Decoder

    - <start>토큰 -> Embedding->LSTM + context -> 단어예측

    - 예측단어 -> 다음입력 ->반복(<end> 나올때 까지)

In [3]:
# 2-1  Encoder 클래스
import tensorflow as tf

class Encoder(tf.keras.Model):
    '''
    입력 문장을 고차원 벡터로 압축
    Embedding -> LSTN -> Hidden States 출력
    구조 : 입력 (정수시퀀스) -> Embedding -> LSTM -> 모든 타임스텝의 출력
    '''
    def __init__(self, vocab_size , embedding_dim , enc_units , batch_size):
        '''
        Args:
            vocab_size : 단어사전 크기
            embedding_dim : 임베딩 벡터 차원
            enc_units : LMS units 수 (hidden state 차원)
            batch_size : 배치 크기
        '''
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        
        # Embedding Layers : 정수 -> 밀집벡터(Dense Vetor) : 작은 차원 대부분의 값이 0이 아님, 연속된 실수값 공간 효율성
        # 학습 과정을 통해 단어 간 의미적 관계 학습한다 ~ 이 말은 유사한 단어는 가까운 벡터 공간에 위치한다~
        # 단점은 훈련이 필요하고 사전학습된 모델을 사용하거나 임베딩을 학습한다.
        # 밀집벡터 하면 희소벡터가 나옴.
        # 희소벡터 : 대부분의 값이 0 / 대표적으로 원-핫벡터 , 차원이 커지고 단어간의 충돌이 없다. 즉, 단어간의 유사도 표현을 못함.
        # 밀집벡터 : 학습기반이고 Word2Vec , Glove , Embedding)
        # 희소벡터 : 규칙기반이고 one-hot , Bow - 초창기 자연어 모델임.
        
        # mask_zero = True  패딩에 마스크처리를 해서 모델이 해석하지 않게 한다.
        self.embedding = tf.keras.layers.Embedding(
            vocab_size, embedding_dim , mask_zero = True
        )
        self.lstm = tf.keras.layers.LSTM(
            enc_units,
            return_sequences = True, # [batch , seq_len , units] 형태가 나올꺼임
            return_state = True, # (output , h , c)
            recurrent_initializer = 'glorot_uniform' #가중치 초기화
        )
    def call(self, x, hidden):
        '''
        입력 시퀀스를 처리해서 hidden states 생성
        Args:
            x : 입력 시퀀스 [ batch_size, seq_len]
            hidden : 초기 hidden state(첫 호출시 0벡터)
        Returns:
            output: 모든타임스템프의 출력   [batch, seq_le, enc_units]
            state_h : 마지막 hidden state   [batch, enc_units]
            state_c : 마지막 cell state [batch,enc_units]
        '''
        # 1단계 Embedding
        # [batch, seq_len]-> [batch,seq_len, embedding_dim]
        x = self.embedding(x)

        # 2단계 : LSTM 처리
        # output : 모든타임스텝의 출력(Attenton에서 사용)
        # state_h : 마지막 hidden state(디코더 초기화)
        # state_c : 마지막 cell state(LSTM 내부 메모리)
        output, state_h ,state_c =  self.lstm(x,initial_state=hidden)
        return output,state_h,state_c
    def initialize_hidden_state(self):
        '''
        인코더의 초기 hidden state 0으로 초기화
        Returns:
            두개의 0텐서 [ batch, enc_unit] (h와c)
        '''
        return [
            tf.zeros((self.batch_size, self.enc_units  )),
            tf.zeros((self.batch_size, self.enc_units  ))
        ]
# 2-2 Encoder 테스트
embedding_dim = 2
units = 512  #LSTM unit 수
# encoder 객체
encoder = Encoder(korean_vocab_size,embedding_dim,units,BATCH_SIZE)

#샘플 데이터
sample_input_batch, sample_target_batch =  next(iter(dataset))
inital_hidden = encoder.initialize_hidden_state()

# encorder 클래스에 forward-> call 호출해서 결과를 얻어야.... why call??  우리는 모델을 funtional API 방식
# 객체를 함수처럼 사용하면 된다.
sample_output, sample_h, sample_c =  encoder(sample_input_batch, inital_hidden )

print(f'encoder 출력형태')
print(f'입력형태 {sample_input_batch.shape}')  # [batch, korean_seq_len]
print(f'출력형태 {sample_target_batch.shape}') # [batch, english_seq_len] ]
print(f'hidden state :{sample_h.shape}')        # [batch, units]
print(f'cell state :{sample_c.shape}')      # [batch, units]

units

encoder 출력형태
입력형태 (2, 6)
출력형태 (2, 7)
hidden state :(2, 512)
cell state :(2, 512)


512

- 인코더의 출력형태
    - 입력 형태 (2,3)
    - 출력 형태 (2,5,512) ------> 모든 타임스텝의 hidden state(Attention에서 사용)
    - hidden state (2,512) ---> 마지막 타임 스템프의 상태 (디코더 초기화)
    - cell state (2,512) -----> LSTM 내부 메모리

# -------- Step 3 ------

### Attention Layer 구현

1. Query 생성: 디코더의 현재 hidden state
2. Key/Value: 인코더의 모든 hidden states
3. Score 계산: Q · K^T (내적으로 유사도 측정)
4. Attention Weight: Softmax(Score) (확률 분포로 변환)
5. Context Vector: Weight로 Value를 가중합

In [4]:
# query,key,value,attention score, context vector
class BAttention(tf.keras.layers.Layer):
    '''
    query : 무엇을 찾고 있는지(디코더의 현재 상태)
    key : 어디에 정보가 있는지(인코더의 각 타임스텝)
    value : 가저올 내용(인코더의 실제 정보)
    '''
    def __init__(self, units):
        '''
        Args:
            units : Attention레이어의 차원(보통 인코더 units와 동일)
        '''
        super(BAttention,self).__init__()
        # query 변환행렬 디코더의 hidden state를 attenton 공간
        self.W1 = tf.keras.layers.Dense(units)
        # key 변환행렬 인코더의 hidden state를 attenton 공간
        self.W2 = tf.keras.layers.Dense(units)
        # v : score 계산용 벡터
        self.V = tf.keras.layers.Dense(1)
    def call(self, query, values):
        """
        Args:
            query: 디코더의 hidden state [batch, dec_units]
            values: 인코더의 모든 hidden states [batch, enc_seq_len, enc_units]
        
        Returns:
            context_vector: 가중합된 인코더 정보 [batch, enc_units]
            attention_weights: 각 타임스텝의 가중치 [batch, enc_seq_len, 1]
        
        동작 과정:
            1. Query와 Key를 같은 차원으로 변환
            2. Score 계산 (유사도 측정)
            3. Softmax로 확률 분포 생성
            4. 가중합으로 context vector 생성
        """
        
        # 1단계: Query 확장
        # [batch, dec_units] → [batch, 1, dec_units]
        # 이유: Key와 브로드캐스팅하기 위해 차원 추가
        query_with_time_axis = tf.expand_dims(query, 1)
        
        # 2단계: Score 계산 (Bahdanau 방식)
        # W1(query): [batch, 1, units]
        # W2(values): [batch, enc_seq_len, units]
        # 덧셈 브로드캐스팅: [batch, enc_seq_len, units]
        # tanh: 비선형 활성화 (-1 ~ 1 범위)
        # V: [batch, enc_seq_len, 1] (각 타임스텝의 점수)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)
        ))
        
        # 3단계: Attention Weights 계산
        # Softmax: 점수를 확률 분포로 변환 (합이 1)
        # [batch, enc_seq_len, 1]
        # 높은 점수 → 높은 가중치 (더 많이 참고)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # 4단계: Context Vector 계산
        # attention_weights: [batch, enc_seq_len, 1]
        # values: [batch, enc_seq_len, enc_units]
        # 곱셈: [batch, enc_seq_len, enc_units]
        # sum: [batch, enc_units] (가중합)
        # 의미: 중요한 타임스텝의 정보를 더 많이 가져옴
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights
attention_layer =  BAttention(units)    

#query : 디코더의 현재 상태(첫번째 디코딩 스텝)
sample_query = sample_h  
# values : 인코더의 모든 출력
sample_values = sample_output

context_vector, attention_weights =  attention_layer(sample_query, sample_values)
print(f'Attention Weight 첫번재 샘플')
weight_example = attention_weights[0].numpy().flatten()
print(f'가중치 : {weight_example}')
print(f'합계 : {np.sum(weight_example):.4f}')

Attention Weight 첫번재 샘플
가중치 : [0.1666244  0.16667177 0.16668837 0.16666275 0.16667324 0.16667953]
합계 : 1.0000


## -----step 4-------

### Context Vector 결합 및 디코더 구현

Decoder 각 타임스텝:
1. 입력 단어 → Embedding
2. Attention으로 Context Vector 계산
3. Embedding + Context Vector → 결합
4. LSTM으로 처리 → Hidden State 업데이트
5. Dense Layer → 다음 단어 확률 분포

In [5]:
class Decoder(tf.keras.Model):
    """
    목적: Attention 메커니즘을 사용하는 디코더
    개념:
        - Teacher Forcing: 학습 시 정답을 입력으로 사용
        - Attention Context: 매 스텝마다 인코더 정보 참고
        - Output Layer: 단어 확률 분포 생성
    
    일반 Decoder vs Attention Decoder:
        - 일반: 인코더의 마지막 상태만 사용 (정보 손실)
        - Attention: 모든 인코더 상태를 동적으로 참고 (정보 보존)
    """
    
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        """
        Args:
            vocab_size: 출력 단어 사전 크기
            embedding_dim: 임베딩 벡터 차원
            dec_units: LSTM 유닛 수
            batch_sz: 배치 크기
        """
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        
        # Embedding Layer: 출력 단어를 벡터로 변환
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        # LSTM Layer: 시퀀스 생성
        # return_sequences=True: 다음 타임스텝을 위해 출력 유지
        # return_state=True: hidden state 업데이트
        self.lstm = tf.keras.layers.LSTM(
            dec_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform'
        )
        
        # Output Layer: 단어 확률 분포 생성
        # vocab_size 차원 → 각 단어의 로그 확률
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # Attention Layer
        self.attention = BAttention(dec_units)
    
    def call(self, x, hidden, enc_output):
        """
        목적: 한 타임스텝의 디코딩 수행
        
        Args:
            x: 입력 단어 [batch, 1] (Teacher Forcing)
            hidden: 이전 타임스텝의 hidden state [batch, dec_units]
            enc_output: 인코더의 모든 출력 [batch, enc_seq_len, enc_units]
        
        Returns:
            predictions: 다음 단어 확률 분포 [batch, vocab_size]
            state_h: 현재 hidden state [batch, dec_units]
            attention_weights: Attention 가중치 [batch, enc_seq_len, 1]
        
        동작 과정:
            입력 → Embedding → Attention → 결합 → LSTM → 출력
        """
        
        # 1단계: Attention으로 Context Vector 계산
        # hidden: [batch, dec_units] (Query)
        # enc_output: [batch, enc_seq_len, enc_units] (Key, Value)
        context_vector, attention_weights = self.attention(hidden, enc_output)
        
        # 2단계: 입력 단어 임베딩
        # [batch, 1] → [batch, 1, embedding_dim]
        x = self.embedding(x)
        
        # 3단계: 임베딩과 Context Vector 결합
        # context_vector: [batch, enc_units] → [batch, 1, enc_units]
        # x: [batch, 1, embedding_dim]
        # 결합: [batch, 1, embedding_dim + enc_units]
        # 의미: 현재 입력 + 인코더 정보를 함께 사용
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # 4단계: LSTM 처리
        # output: [batch, 1, dec_units]
        # state_h, state_c: [batch, dec_units]
        output, state_h, state_c = self.lstm(x)
        
        # 5단계: 차원 조정 및 출력 생성
        # [batch, 1, dec_units] → [batch, dec_units]
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # 6단계: 단어 확률 분포 생성
        # [batch, dec_units] → [batch, vocab_size]
        # Softmax는 loss 함수에서 처리 (수치 안정성)
        predictions = self.fc(output)
        
        return predictions, state_h, attention_weights
    
decoder = Decoder(english_vocab_size, embedding_dim,units,BATCH_SIZE)
# 샘플 입력 (디코더의 첫 번째 입력은 <start> 토큰)
sample_decoder_input = tf.random.uniform((BATCH_SIZE, 1))  # [batch, 1]
sample_decoder_hidden = sample_h  # 인코더의 마지막 hidden state

predictions, dec_h, attn_weights = decoder(
    sample_decoder_input,
    sample_decoder_hidden,
    sample_output
)
print('Decoder 출력 형태')
print(f'입력형태 : {sample_decoder_input.shape}')
print(f'predictions 형태 : {predictions.shape}')  # batch, vocab_size-->english_vocab_size
print(f'Hidden state 형태 : {dec_h.shape}')
print(f'attn_weights state 형태 : {attn_weights.shape}')

predictions[0, :5]  # 첫 5개 단어 점수

Decoder 출력 형태
입력형태 : (2, 1)
predictions 형태 : (2, 33)
Hidden state 형태 : (2, 512)
attn_weights state 형태 : (2, 6, 1)


<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([-9.9978992e-05, -7.3953110e-05, -1.0391870e-04,  1.2256718e-05,
       -3.3448912e-05], dtype=float32)>

## -------step5 -----

Teacher Forcing:

학습 시: 입력으로 정답 사용 (빠른 수렴)
  디코더 입력: <start> → 정답단어1 → 정답단어2 → ...

추론 시: 입력으로 이전 예측 사용

  디코더 입력: <start> → 예측단어1 → 예측단어2 → ...

Loss 계산:
1. 예측 분포와 정답 비교 (Cross Entropy)
2. 패딩은 무시 (마스킹)
3. 역전파로 가중치 업데이트

In [6]:
def evaluate(sentence):
    """
	한국어 문장을 영어로 번역
    
    Args:
        sentence: 입력 문장 (한국어)
    
    Returns:
        result: 번역된 문장 (영어)
        attention_weights: Attention 가중치 (시각화용)
    
    Greedy Decoding:
        매 스텝마다 가장 높은 확률의 단어 선택
        빠르지만 최적해 보장 안 됨
        
    Beam Search (고급):
        여러 후보를 동시에 탐색
        더 좋은 번역 가능하지만 느림
    """
    
    # 1단계: 문장 전처리
    sentence = preprocess_sentence(sentence, is_korean=True)
    
    # 2단계: 정수 시퀀스로 변환
    inputs = [korean_tokenizer.word_index.get(word, korean_tokenizer.word_index['<unk>'])
            for word in sentence.split()]
    
    # 3단계: 패딩 추가
    inputs = tf.keras.preprocessing.sequence.pad_sequences(
        [inputs],
        maxlen=max_korean_len,
        padding='post'
    )
    inputs = tf.convert_to_tensor(inputs)  # [1, seq_len]
    
    result = ''
    attention_plot = np.zeros((max_english_len, max_korean_len))
    
    # 4단계: Encoder 실행
    hidden = [tf.zeros((1, units)), tf.zeros((1, units))]
    enc_out, enc_h, enc_c = encoder(inputs, hidden)
    
    # 5단계: Decoder 초기화
    dec_hidden = enc_h
    dec_input = tf.expand_dims([english_tokenizer.word_index['<start>']], 0)
    
    # 6단계: 디코딩 루프
    for t in range(max_english_len):
        # 예측 수행
        predictions, dec_hidden, attention_weights = decoder(
            dec_input,
            dec_hidden,
            enc_out
        )
        
        # Attention 가중치 저장 (시각화용)
        attention_weights = tf.reshape(attention_weights, (-1,))
        attention_plot[t] = attention_weights.numpy()
        
        # 가장 높은 확률의 단어 선택 (Greedy)
        predicted_id = tf.argmax(predictions[0]).numpy()
        
        # 정수 → 단어 변환
        predicted_word = english_tokenizer.index_word.get(predicted_id, '<unk>')
        
        # <end> 토큰이면 중단
        if predicted_word == '<end>':
            break
        
        # 결과에 추가
        result += predicted_word + ' '
        
        # 예측 단어를 다음 입력으로 사용 (Teacher Forcing 없음)
        dec_input = tf.expand_dims([predicted_id], 0)
    
    return result.strip(), attention_plot

In [7]:
korean_sentences[1]

'오늘 날씨가 좋아요'

In [13]:
# 번역 테스트 
for kor in korean_sentences:
    print(f'한국어 : {kor:60s}  번역 : {evaluate(kor)[0]}')


# evaluate(korean_sentences[1])

한국어 : 안녕하세요                                                         번역 : he don’t he don’t he don’t he
한국어 : 오늘 날씨가 좋아요                                                    번역 : he don’t the he don’t the he
한국어 : 드라마 보고 있어                                                     번역 : i’m watching coffee don’t coffee don’t coffee
한국어 : 이것은 사과입니다                                                     번역 : he don’t the he don’t the he
한국어 : 고양이가 자고 있어요                                                   번역 : he don’t coffee watching the he don’t
한국어 : 내일 비가 올까요                                                     번역 : he don’t it feel feel feel feel
한국어 : 저는 커피를 좋아해요                                                   번역 : he don’t coffee watching the he don’t
한국어 : 그는 의사입니다                                                      번역 : he don’t the he don’t the he
한국어 : 이 책은 재미있어요                                                    번역 : he don’t it feel feel feel feel
한국어 : 오늘 몸이 안 좋아                        